<a href="https://colab.research.google.com/github/QWL55/Predicting_Migration_Flow/blob/main/notebook/CS224W_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS224W - Final Project**

# Setup
First let us check which version of PyTorch you are running

In [1]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 2.4.0+cu121


In [ ]:
#!pip install torch==2.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# Install torch geometric
torch_version = str(torch.__version__)
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src
!pip install torch-sparse -f $sparse_src
!pip install torch-geometric
!pip install ogb

Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 70.2 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.7 MB/s eta 0:00:00


# 1) GNN: Binary Link Prediction

In this section we will build our naive baseline graph neural network using PyTorch Geometric. Then we will apply it to the task of predicting migration flow (binary link prediction).

Specifically, we will use GCN as the foundation for the graph neural network ([Kipf et al. (2017)](https://arxiv.org/pdf/1609.02907.pdf)). To do so, we will work with PyG's built-in `GCNConv` layer.

## Setup

In [77]:
import torch
import pandas as pd
import torch.nn.functional as F
print(torch.__version__)
from sklearn.metrics import roc_auc_score
# The PyG built-in GCNConv
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
import torch_geometric.transforms as T
#from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

2.4.0+cu121


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
# change the path to your project folder
os.chdir('/content/drive/MyDrive/Stanford SOC/CS224W/final_projects')

In [41]:
os.listdir()

['CS224W_Colab_2.ipynb',
 'CS224W_Final_Projects.ipynb',
 'county_flow.csv',
 'dataset',
 'county_node_info.csv']

## Load and Preprocess the Dataset

In [94]:
import pandas as pd
import torch
from torch_geometric.data import Data

# Load the datasets
county_flow_df = pd.read_csv('county_flow.csv', dtype={"fips_orig": str, "fips_dest": str})
county_node_df = pd.read_csv('county_node_info.csv', dtype={"GEOID": str})

# Step 1: Create a mapping from GEOID to index and
# map fips_orig and fips_dest in county_flow_df to indices
geo_to_index = {geo_id: idx for idx, geo_id in enumerate(county_node_df['GEOID'])}

county_flow_df['src'] = county_flow_df['fips_orig'].map(geo_to_index)
county_flow_df['dst'] = county_flow_df['fips_dest'].map(geo_to_index)

# Drop any rows with missing mappings (i.e., if a fips code doesn't exist in county_node_df)
county_flow_df = county_flow_df.dropna(subset=['src', 'dst']).astype({'src': int, 'dst': int})

# Step 2: Sort each edge to treat (src, dst) and (dst, src) as the same
county_flow_df['min_index'] = county_flow_df[['src', 'dst']].min(axis=1)
county_flow_df['max_index'] = county_flow_df[['src', 'dst']].max(axis=1)

# Step 3: Drop duplicates based on sorted pairs (min_index, max_index)
unique_edges_df = county_flow_df.drop_duplicates(subset=['min_index', 'max_index'])

# Step 4: Extract the final edge list
edge_index_df = unique_edges_df[['min_index', 'max_index']]

# Convert edge list to PyTorch tensor
edge_index = torch.tensor(edge_index_df.values.T, dtype=torch.long)

# Convert county_node_df (excluding GEOID) to a PyTorch tensor for node features
county_node_features = county_node_df.drop(columns=['GEOID'], errors='ignore').values
county_node_features = torch.tensor(county_node_features, dtype=torch.float)

# Create PyG Data object
data = Data(x=county_node_features, edge_index=edge_index)

# Display data to confirm
print(data)


Data(x=[3142, 9], edge_index=[2, 24495])


In [104]:
transform = T.Compose([T.ToSparseTensor(), T.RandomLinkSplit(is_undirected=True)])

train_data, val_data, test_data = transform(data)
print(train_data)

Data(x=[3142, 9], adj_t=[3142, 3142, nnz=24495], edge_index=[2, 34294], edge_label=[34294], edge_label_index=[2, 34294])


In [103]:
train_data.adj_t

SparseTensor(row=tensor([   1,    7,    9,  ..., 3139, 3139, 3140]),
             col=tensor([   0,    1,    7,  ..., 3119, 3137, 3127]),
             size=(3142, 3142), nnz=24495, density=0.25%)

## GCN Model

Now we will implement our GCN model!

Please follow the figure below to implement the `forward` function.


![test](https://drive.google.com/uc?id=128AuYAXNXGg7PIhJJ7e420DoPWKb-RtL)

In [107]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels))

        # Add hidden GCNConv layers (hidden to hidden)
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))

        for _ in range(num_layers - 1):
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))

        # Add the final GCNConv layer (hidden to output)
        self.convs.append(GCNConv(hidden_channels, out_channels))
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def encode(self, x, edge_index):

        for i in range(len(self.convs) - 1):
            if i == 0:
                out = self.convs[i](x, edge_index)
            else:
                out = self.convs[i](out, edge_index)
            out = self.bns[i](out)
            out = F.relu(out)
            out = F.dropout(out, p=self.dropout, training=self.training)

        out = self.convs[-1](out, edge_index)

        return out

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [108]:
def get_link_labels(pos_edge_index, neg_edge_index):
    # returns a tensor:
    # [1,1,1,1,...,0,0,0,0,0,..] with the number of ones is equel to the lenght of pos_edge_index
    # and the number of zeros is equal to the length of neg_edge_index
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.

    return link_labels


def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()

    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


In [111]:
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 200,
}

model = GCN(data.num_features, args['hidden_dim'], 64,
            args['num_layers'], args['dropout']).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=args['lr'])
criterion = torch.nn.BCEWithLogitsLoss()

In [112]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
    #       f'Test: {test_auc:.4f}')
    log = 'Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    if epoch % 10 == 0:
        print(log.format(epoch, loss, val_auc, test_auc))

Epoch: 010, Loss: 2.8140, Val: 0.8443, Test: 0.8504
Epoch: 020, Loss: 1.1383, Val: 0.8902, Test: 0.8968
Epoch: 030, Loss: 0.7304, Val: 0.8935, Test: 0.9011
Epoch: 040, Loss: 0.6666, Val: 0.8926, Test: 0.9001
Epoch: 050, Loss: 0.6329, Val: 0.8775, Test: 0.8878
Epoch: 060, Loss: 0.6198, Val: 0.8453, Test: 0.8555
Epoch: 070, Loss: 0.6152, Val: 0.8515, Test: 0.8626
Epoch: 080, Loss: 0.6131, Val: 0.8424, Test: 0.8540
Epoch: 090, Loss: 0.6126, Val: 0.8483, Test: 0.8596
Epoch: 100, Loss: 0.6136, Val: 0.8495, Test: 0.8609
